# FireRoad053 Model

In [1]:
## if you have not installed package "CSV" or "JSON", please uncomment the line below and execute it
# using Pkg; Pkg.add("CSV")

using DataFrames, CSV
student1 = CSV.read("finaldata/student1.csv", DataFrame)
student2 = CSV.read("finaldata/student2.csv", DataFrame)
student3 = CSV.read("finaldata/student3.csv", DataFrame)
student4 = CSV.read("finaldata/student4.csv", DataFrame)
student5 = CSV.read("finaldata/student5.csv", DataFrame)
student6 = CSV.read("finaldata/student6.csv", DataFrame)
reqs = CSV.read("finaldata/REQ.csv", DataFrame)
reqnots = CSV.read("finaldata/REQNOT.csv", DataFrame)
reqsems = CSV.read("finaldata/REQSEM.csv", DataFrame)
classes = CSV.read("finaldata/parsedsp21_actual_classes.csv", DataFrame)

,Column1,gir_attribute,hass_attribute,id,prereqs
,String,String?,String?,String,String?
1,8.01,PHY1,missing,8.01,missing
2,8.02,PHY2,missing,8.02,GIR:CAL1 and GIR:PHY1
3,18.01,CAL1,missing,18.01,missing
4,18.02,CAL2,missing,18.02,GIR:CAL1
5,7.014,BIOL,missing,7.014,missing
6,7.016,BIOL,missing,7.016,missing
7,3.091,CHEM,missing,3.091,missing
8,5.111,CHEM,missing,5.111,missing
9,5.07,REST,missing,5.07,5.12


In [2]:
#classes[!,"sections"][3]
using JSON
classesjson = JSON.parsefile("finaldata/combodict.json")
#classesjson = JSON.parsefile("finaldata/parsedsp21_dummy.json") #multiple sections
# one section: finaldata/parsedsp21_actual_classes.json
sections = ["LectureSession", "RecitationSession", "LabSession"]
combInd = [1, 8, 26, 27, 37, 48, 59, 70, 79, 82, 94, 102, 104, 106, 108, 116, 124, 127, 139, 161, 173, 177, 178, 502, 510, 524, 534, 546, 547, 548, 549, 551, 557, 558, 561, 562, 564, 565, 566, 576, 579, 581, 582, 583, 584, 585, 587, 589, 590, 600, 603]

# for class in keys(classesjson)
#     for section in keys(classesjson[class]["sections"])
#         print(classesjson[class]["sections"][section])
#     end
# end

#classesjson["6.006"]["sections"]["RecitationSession"]
size(classesjson["6.006"][1])[1]
classesjson["6.006"][1][1]

0.0

In [3]:
indToId = Dict()
for i in 1:size(classes)[1]
    indToId[i] = classes[i,1]
end
indToId

Dict{Any,Any} with 51 entries:
  2  => "8.02"
  11 => "5.13"
  39 => "18.06"
  46 => "24.013"
  25 => "6.042"
  42 => "21H.134"
  29 => "6.819"
  8  => "5.111"
  20 => "6.009"
  14 => "5.602"
  31 => "15.0251"
  33 => "15.068"
  18 => "6.004"
  26 => "6.046"
  35 => "15.312"
  17 => "6.003"
  49 => "9.00"
  44 => "21L.019"
  4  => "18.02"
  37 => "15.780"
  45 => "21M.273"
  13 => "5.601"
  30 => "6.UAT"
  1  => "8.01"
  47 => "21G.701"
  ⋮  => ⋮

In [ ]:
studentdata = JSON.parsefile("finaldata/students_data.json")
studentdata

In [ ]:
classes_more_sections = CSV.read("finaldata/parsedclasses.csv", DataFrame)
classes_more_sections[1, "sections"]

# Model

In [ ]:
#--- Model specification

using JuMP, DataFrames, Gurobi
M = 1000000
S = nrow(classes)
T = 8
O = 150 # number of timeslots
C = 604 # total combinations
min_units = 12 #change this for each student!
max_units = 60 #change this for each student!
min_workload = 20 #change this for each student!
max_workload = 60 #change this for each student!
curr_t = 4 #change this for each student!
model = Model(with_optimizer(Gurobi.Optimizer))

@variable(model, x[1:S, 1:T], Bin)
@variable(model, comb[1:C], Bin)
@objective(model, Max, sum(sum(x[s, t] for t = 1:T) * classes[s, 9] for s = 1:S))

# Constraints

In [ ]:
# 1. Each student takes (or gets credit for) all of the Science/Math GIRs.
# Math GIR
@constraint(model, math, sum(x[3, t] + x[4, t] for t = 1:T) == 2)

# Physics GIR
@constraint(model, physics, sum(x[1, t] + x[2, t] for t = 1:T) == 2)

# Chemistry GIR
@constraint(model, chem, sum(x[7, t] + x[8, t] for t = 1:T) == 1)

# Biology GIR
@constraint(model, bio, sum(x[5, t] + x[6, t] for t = 1:T) == 1)

# 2. Each student takes 8 HASS subjects.
@constraint(model, hass, sum(x[s, t] for s = 42:50 for t = 1:T) >= 8)

# 3. Each student satisfies the requirements of her major (and minor).
# 6-3 Major Requirements
# @constraint(model, major63, sum(x[15, t] + x[25, t] + x[30, t] + x[18, t] + x[19, t] + x[20, t] + x[22, t] + x[23, t] + x[24, t] + x[27, t] + x[28, t] + x[29, t] + x[17, t] for t in 1:T) == 13)
# @constraint(model, major6360x, sum(x[21, t] + x[27, t] for t in 1:T) >= 1)

# # 15-2 Major Requirements
# @constraint(model, major152, sum(x[34, t] + x[15, t] + x[16, t] + x[35, t] + x[32, t] + x[40, t] + x[41, t] + x[37, t] + x[24, t] + x[36, t] + x[38, t] + x[25, t] + x[39, t] + x[31, t] for t in 1:T) == 14)

# #5 Minor Requirements
# @constraint(model, minor5, sum(x[s, t] for s in 9:14 for t in 1:T) == 6)
# @constraint(model, minor5b, sum(x[51, t] for t in 1:T) == 1)

# 4. No subject is taken prior to its prerequisites.
@constraint(model, prereq802[t in 2:T], sum(x[1, j] + x[3, j] for j in 1:t-1) >= 2 * x[2, t])
@constraint(model, prereq1802[t in 2:T], sum(x[3, j] for j in 1:t-1) >= x[4, t])
@constraint(model, prereq503[t in 2:T], sum(x[10, j] for j in 1:t-1) >= x[51, t])
@constraint(model, prereq507[t in 2:T], sum(x[10, j] for j in 1:t-1) >= x[9, t])
@constraint(model, prereq512[t in 2:T], sum(x[7, j] + x[8, j] for j in 1:t-1) >= x[10, t])
@constraint(model, prereq513[t in 2:T], sum(x[10, j] for j in 1:t-1) >= x[11, t])
@constraint(model, prereq5310[t in 2:T], sum(x[10, j] for j in 1:t-1) >= x[12, t])
@constraint(model, prereq5601cal[t in 2:T], sum(x[4, j] for j in 1:t-1) >= x[13, t])
@constraint(model, prereq5601chem[t in 2:T], sum(x[7, j] + x[8, j] for j in 1:t-1) >= x[13, t])
@constraint(model, prereq5602[t in 2:T], sum(x[13, j] for j in 1:t-1) >= x[14, t])
@constraint(model, prereq60002[t in 2:T], sum(x[15, j] for j in 1:t-1) >= x[16, t])
@constraint(model, prereq6003[t in 2:T], sum(x[3, j] + x[15, j] for j in 1:t-1) >= 2 * x[17, t])
@constraint(model, prereq6004[t in 2:T], sum(x[2, j] + x[15, j] for j in 1:t-1) >= 2 * x[18, t])
@constraint(model, prereq6006a[t in 2:T], sum(x[25, j] for j in 1:t-1) >= x[19, t])
@constraint(model, prereq6006b[t in 2:T], sum(x[15, j] + x[20, j] for j in 1:t-1) >= x[19, t])
@constraint(model, prereq6009[t in 2:T], sum(x[15, j] for j in 1:t-1) >= x[20, t])
@constraint(model, prereq603[t in 2:T], sum(x[2, j] + x[4, j] for j in 1:t-1) >= 2 * x[21, t])
@constraint(model, prereq6031[t in 2:T], sum(x[20, j] for j in 1:t-1) >= x[22, t])
@constraint(model, prereq6033[t in 2:T], sum(x[18, j] + x[20, j] for j in 1:t-1) >= 2 * x[23, t])
@constraint(model, prereq6036[t in 2:T], sum(x[4, j] + x[15, j] for j in 1:t-1) >= 2 * x[24, t])
@constraint(model, prereq6042[t in 2:T], sum(x[3, j] for j in 1:t-1) >= x[25, t])
@constraint(model, prereq6046[t in 2:T], sum(x[19, j] for j in 1:t-1) >= x[26, t])
@constraint(model, prereq608[t in 2:T], sum(x[2, j] + x[15, j] for j in 1:t) >= 2 * x[27, t])
@constraint(model, prereq6806[t in 2:T], sum(x[26, j] for j in 1:t-1) >= x[28, t])
@constraint(model, prereq6819[t in 2:T], sum(x[25, j] + x[39, j] for j in 1:t-1) >= 2 * x[29, t])
@constraint(model, prereq150251[t in 2:T], sum(x[50, j] for j in 1:t-1) >= x[31, t])
@constraint(model, prereq15053[t in 2:T], sum(x[15, j] for j in 1:t-1) >= x[32, t])
@constraint(model, prereq1806[t in 2:T], sum(x[4, j] for j in 1:t-1) >= x[39, t])
@constraint(model, prereq18600[t in 2:T], sum(x[4, j] for j in 1:t-1) >= x[40, t])
@constraint(model, prereq18650[t in 2:T], sum(x[40, j] for j in 1:t-1) >= x[41, t])

# 5. Each student takes 216 units beyond GIRs.
@constraint(model, units, sum(x[s, t] * classes[s, 8] for s = 1:S for t = 1:T) >= 208)

# 6. No two subjects taken in Spring 2021 should overlap in time.
# choose one or none combos for each class
@constraint(model, choosecombo[s in 1:S], sum(comb[combInd[s]:combInd[s]+size(classesjson[indToId[s]])[1]-1]) <= 1)

# choose combo for a class iff choose class
@constraint(model, takeclass[s in 1:S], sum(comb[combInd[s]:combInd[s]+size(classesjson[indToId[s]])[1]-1]) - x[s, curr_t] == 0)

# for each timeslot, no conflicts
@constraint(model, conflict[o in 1:O], sum(classesjson[indToId[s]][i][o]*comb[combInd[s]:combInd[s]+size(classesjson[indToId[s]])[1]-1][i] for s in 1:S for i in 1:size(classesjson[indToId[s]])[1]) <= 1)
                         
# 7. Students cannot take the same subject twice.
@constraint(model, once[s in 1:S], sum(x[s, t] for t = 1:T) <= 1)

# 8. Each student can specify if they want to take a specific subject in the future.
@constraint(model, req[s in 1:S], sum(x[s, t] for t in 1:T) >= 1 - M * (1 - reqs[s, 1]))

# 9. Each student can specify which semester they want to take a subject.
@constraint(model, reqsem[s in 1:S, t in 1:T], x[s, t] >= 1 - M * (1 - reqsems[s, t]))

# 10. Each student can specify which semester they do not want to take a subject. 
@constraint(model, reqnot[s in 1:S, t in 1:T], x[s, t] <= 1 + M * (1 - reqnots[s, t]))
    
# 11. The minimum and maximum number of units that each student wants to take each semester.
@constraint(model, minunits[t in 1:T], sum(x[s, t] * classes[s, 8] for s in 1:S) >= min_units)
@constraint(model, maxunits[t in 1:T], sum(x[s, t] * classes[s, 8] for s in 1:S) <= max_units)
    
# 12. The minimum and maximum workload (in number of hours) that each student wants to take on each semester.
@constraint(model, minwork[t in 1:T], sum(x[s, t] * classes[s, 10] for s in 1:S) >= min_workload)
@constraint(model, maxwork[t in 1:T], sum(x[s, t] * classes[s, 10] for s in 1:S) <= max_workload)

# 13. Restricts taking classes to the semester they're offered
@constraint(model, fall_only[s in [1, 9, 11, 35, 37]], sum(x[s, t] for t in [2, 4, 6, 8]) == 0)
@constraint(model, spring_only[s in [6, 21, 23, 27, 28, 29, 31, 32, 33, 38, 51]], sum(x[s, t] for t in [1, 3, 5, 7]) == 0)

In [ ]:
#Classes Students have already taken
# !! Change for each student (change var name and t and file index)

# @constraint(model, student1_taken[s in 1:S, t in 1:1], x[s, t] == student1[s, t])
# @constraint(model, student2_taken[s in 1:S, t in 1:3], x[s, t] == student2[s, t])
# @constraint(model, student3_taken[s in 1:S, t in 1:5], x[s, t] == student3[s, t])

# @constraint(model, student4_taken[s in 1:S, t in 1:1], x[s, t] == student4[s, t])
# @constraint(model, student5_taken[s in 1:S, t in 1:3], x[s, t] == student5[s, t])
# @constraint(model, student6_taken[s in 1:S, t in 1:5], x[s, t] == student6[s, t])


In [ ]:
#--- Write codes here to print your solutions
optimize!(model)
@show termination_status(model)
@show objective_value(model)

In [ ]:
display("text/csv", value.(x)) # could also display JSON

In [ ]:
using CSV; CSV.write("model_results/results_stud1.csv", DataFrame(value.(x)))